# Load the tests to compare

In [1]:
import os
import sys
import json
import pickle
import pandas as pd
__file__ = globals()['_dh'][0]

# get an absolute path to the directory that contains parent files
project_dir = __file__ = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, '..', '..')))

from experiments.utils.constants import TESTS_RESULTS_PATH
series = 64
start_test = 0
end_test = 10
tests_ids = [i for i in range(start_test, end_test)]
tests_names = [i for i in range(start_test, end_test)]

tests = {}


for test_id, test_name in zip(tests_ids, tests_names):
    test_path = os.path.join(
        TESTS_RESULTS_PATH,
        'series',
        str(series),
        'tests',
        str(test_id))
    with open(os.path.join(test_path, 'info.json')) as cf:
        info = json.loads(cf.read())
    with open(os.path.join(test_path, 'episodes.pickle'), 'rb') as input_file:
        episodes = pickle.load(input_file)
    tests.update({test_name: {
        'info': info,
        'episodes': episodes
    }})
test_types = [test['info']['algorithm'] for _, test in tests.items()]
print("availeble tests:\n {}\ntests_types:\n {}".format(
    list(tests.keys()),
    test_types))

availeble tests:
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
tests_types:
 ['PPO', 'PPO', 'PPO', 'PPO', 'PPO', 'PPO', 'PPO', 'PPO', 'PPO', 'PPO']


# Necessary stats of a test

In [2]:
final_stats = []
for test_id in range(start_test, end_test):
    users_distances_episodes_average = [episode['users_distances'].mean() for episode in tests[test_id]['episodes']]
    users_distances_test_average = sum(users_distances_episodes_average) / len(users_distances_episodes_average)
    num_consolidated_episodes_average = [episode['num_consolidated'].mean() for episode in tests[test_id]['episodes']]
    num_consolidated_test_average = sum(num_consolidated_episodes_average) / len(num_consolidated_episodes_average)
    # print(users_distances_episodes_average)
    # print(users_distances_test_average)
    # print(num_consolidated_episodes_average)
    # print(num_consolidated_test_average)
    final_stats.append({
        'test_id': test_id,
        'checkpoint': tests[test_id]['info']['checkpoint'],
        'network_id': tests[test_id]['info']['network_id'],
        'trace_id_test': tests[test_id]['info']['trace_id_test'],
        'experiments': tests[test_id]['info']['experiments'],
        'penalty_latency': tests[test_id]['info']['penalty_latency'],
        'penalalty_consolidated': tests[test_id]['info']['penalty_consolidated'],
        'users_distances_test_average': users_distances_test_average,
        'num_consolidated_test_average': num_consolidated_test_average
    })

final_stats_df = pd.DataFrame(final_stats)
final_stats_df['test_types'] = test_types
# final_stats_df.loc[df['penalalty_consolidated']]

# TODO add the following information to the table (from the info.json file)
# - penatly_latency
# - penalty_conslidation

In [3]:
tests[test_id]['info']

{'series': 64,
 'dataset_id': 6,
 'workload_id': 0,
 'network_id': 1,
 'trace_id': 2,
 'trace_id_test': 2,
 'checkpoint': 1000,
 'experiment_str': 'PPO_SimEdgeEnv_01b4e_00000_0_2021-12-09_22-33-18',
 'experiments': 4,
 'episode_length': 10,
 'num_episodes': 5,
 'algorithm': 'PPO',
 'penalty_latency': 10,
 'penalty_consolidated': 2}

# compare tests - tables

In [4]:
final_stats_df

,test_id,checkpoint,network_id,trace_id_test,experiments,penalty_latency,penalalty_consolidated,users_distances_test_average,num_consolidated_test_average,test_types
0,0,1000,1,0,0,2,2,1.344161,5.00,PPO
1,1,1000,1,0,1,4,2,1.030074,5.00,PPO
2,2,1000,1,0,2,6,2,1.408830,4.88,PPO
3,3,1000,1,0,3,8,2,0.979019,5.42,PPO
4,4,1000,1,0,4,10,2,1.064517,5.82,PPO
5,5,1000,1,2,0,2,2,1.171253,5.00,PPO
6,6,1000,1,2,1,4,2,0.926151,5.00,PPO
7,7,1000,1,2,2,6,2,1.221719,4.84,PPO
8,8,1000,1,2,3,8,2,1.031852,5.42,PPO
9,9,1000,1,2,4,10,2,0.995600,5.80,PPO


In [5]:
final_stats_df[(final_stats_df['experiments'] == 0) & (final_stats_df['trace_id_test'] == 2)].sort_values(by=['num_consolidated_test_average'])

,test_id,checkpoint,network_id,trace_id_test,experiments,penalty_latency,penalalty_consolidated,users_distances_test_average,num_consolidated_test_average,test_types
5,5,1000,1,2,0,2,2,1.171253,5.0,PPO


# compare tests - plots

In [18]:
final_stats_df[(final_stats_df['experiments'] == 4) & (final_stats_df['trace_id_test'] == 2)].sort_values(by=['users_distances_test_average'])

,test_id,checkpoint,network_id,trace_id_test,experiments,penalty_latency,penalalty_consolidated,users_distances_test_average,num_consolidated_test_average,test_types
35,35,1000,1,2,4,0,10,0.634013,4.00,PPO
17,17,500,1,2,4,0,10,0.654679,3.62,PPO


In [104]:
final_stats_df[(final_stats_df['network_id'] == 7) & (final_stats_df['trace_id_test'] == 2)].sort_values(by=['users_distances_test_average']) #& (


,test_id,network_id,trace_id_test,experiments,penalty_latency,penalalty_consolidated,users_distances_test_average,num_consolidated_test_average,test_types
32,32,7,2,1,10,2,2.339188,5.640892,PPO
27,27,7,2,0,10,0,2.342369,4.716768,PPO
38,38,7,2,3,2,10,3.173771,0.895337,PPO
39,39,7,2,4,0,10,3.499669,4.999305,PPO
36,36,7,2,2,10,10,4.172751,4.466319,PPO
